## Исследование объявлений о продаже квартир

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет.

### Цель проекта

Научиться определять рыночную стоимость объектов недвижимости. Наша задача — установить параметры.
Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность

Провести следующие исследования:

   -  Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.
   -  Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
   -  Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.
   -  Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.
   -  Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.
   -  Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
   -  Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всему городу?

### Описание данных

| Название столбца     | Описание                                                         |
|----------------------|------------------------------------------------------------------|
| airports_nearest     | расстояние до ближайшего аэропорта в метрах (м)                  |
| balcony              | число балконов                                                   |
| ceiling_height       | высота потолков (м)                                              |
| cityCenters_nearest  | расстояние до центра города (м)                                  |
| days_exposition      | сколько дней было размещено объявление (от публикации до снятия) |
| first_day_exposition | дата публикации                                                  |
| floor                | этаж                                                             |
| floors_total         | всего этажей в доме                                              |
| is_apartment         | апартаменты (булев тип)                                          |
| kitchen_area         | площадь кухни в квадратных метрах (м²)                           |
| last_price           | цена на момент снятия с публикации                               |
| living_area          | жилая площадь в квадратных метрах (м²)                           |
| locality_name        | название населённого пункта                                      |
| open_plan            | свободная планировка (булев тип)                                 |
| parks_around3000     | число парков в радиусе 3 км                                      |
| parks_nearest        | расстояние до ближайшего парка (м)                               |
| ponds_around3000     | число водоёмов в радиусе 3 км                                    |
| ponds_nearest        | расстояние до ближайшего водоёма (м)                             |
| rooms                | число комнат                                                     |
| studio               | квартира-студия (булев тип)                                      |
| total_area           | площадь квартиры в квадратных метрах (м²)                        |
| total_images         | число фотографий квартиры в объявлении                           |

Пояснение: апартаменты — это нежилые помещения, которые не относятся к жилому фонду, но имеют необходимые условия для проживания.

In [43]:
# импортируем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pymystem3 import Mystem
lemm = Mystem().lemmatize # создаем объект лемматизатора

from nltk import SnowballStemmer
stemm = SnowballStemmer('russian').stem # создаем объект стеммера

In [44]:
# загрузим наши данные
data = pd.read_csv('../data/real_estate_data.csv', sep='\t')

# посмотрим информацию о колонках
display(data.head(40))
display(data.info())
display(data.describe())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

None

,total_images,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,kitchen_area,balcony,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
count,23699.000000,2.369900e+04,23699.000000,23699.000000,14504.000000,23613.000000,21796.000000,23699.000000,21421.000000,12180.000000,18157.000000,18180.000000,18181.000000,8079.000000,18181.000000,9110.000000,20518.000000
mean,9.858475,6.541549e+06,60.348651,2.070636,2.771499,10.673824,34.457852,5.892358,10.569807,1.150082,28793.672193,14191.277833,0.611408,490.804555,0.770255,517.980900,180.888634
std,5.682529,1.088701e+07,35.654083,1.078405,1.261056,6.597173,22.030445,4.885249,5.905438,1.071300,12630.880622,8608.386210,0.802074,342.317995,0.938346,277.720643,219.727988
min,0.000000,1.219000e+04,12.000000,0.000000,1.000000,1.000000,2.000000,1.000000,1.300000,0.000000,0.000000,181.000000,0.000000,1.000000,0.000000,13.000000,1.000000
25%,6.000000,3.400000e+06,40.000000,1.000000,2.520000,5.000000,18.600000,2.000000,7.000000,0.000000,18585.000000,9238.000000,0.000000,288.000000,0.000000,294.000000,45.000000
50%,9.000000,4.650000e+06,52.000000,2.000000,2.650000,9.000000,30.000000,4.000000,9.100000,1.000000,26726.000000,13098.500000,0.000000,455.000000,1.000000,502.000000,95.000000
75%,14.000000,6.800000e+06,69.900000,3.000000,2.800000,16.000000,42.300000,8.000000,12.000000,2.000000,37273.000000,16293.000000,1.000000,612.000000,1.000000,729.000000,232.000000
max,50.000000,7.630000e+08,900.000000,19.000000,100.000000,60.000000,409.700000,33.000000,112.000000,5.000000,84869.000000,65968.000000,3.000000,3190.000000,3.000000,1344.000000,1580.000000


Мы видим что у нас есть пропуски в следующих колонках:
- ceiling_height
- floors_total
- living_area
- is_apartment
- kitchen_area
- balcony
- locality_name
- airports_nearest

Также из информации о колонках можно сделать следующие выводы:
1. в колонке total_images тип int64, хотя диапазон чисел в этой колонке от 0 до 50 - можем изменить тип на int8
2. в колонке total_area тип float64, хотя диапазон чисел в этой колонке от 12 до 900 - можем изменить тип на float16
3. в колонке first_day_exposition необходимо преобразовать данные в datatype
4. в колонке rooms тип int64, хотя диапазон чисел в этой колонке от 0 до 19 - можем изменить тип на int8, также подозрительным выглядит значение 0 в данной колонке
5. в колонке ceiling_height тип float64, хотя диапазон чисел в этой колонке от 1 до 100 - можем изменить тип на float8, также подозрительным выглядит значение 1 в данной колонке
6. в колонке floors_total тип float64, хотя диапазон чисел в этой колонке от 1 до 60 и хранятся по факту целочисленные значения - можем изменить тип на int8
7. в колонке living_area тип float64, хотя диапазон чисел в этой колонке от 2 до 409.7 - можем изменить тип на float16
8. в колонке floor тип int64, хотя диапазон чисел в этой колонке от 1 до 33 - можем изменить тип на int8
9. в колонке is_apartment тип object, хотя в колонке хранятся булевы значения - можем изменить тип на bool
10. в колонке kitchen_area тип float64, хотя диапазон чисел в этой колонке от 1.3 до 112 - можем изменить тип на float8
11. в колонке balcony тип float64, хотя диапазон чисел в этой колонке от 0 до 5 и хранятся по факту целочисленные значения - можем изменить тип на int8
12. в колонке locality_name данные необходимо привести к нижнему регистру
13. в колонке airports_nearest тип float64, хотя диапазон чисел в этой колонке от 0 до 84869 - можем изменить тип на float32, также подозрительным выглядит значение 0 в данной колонке
14. в колонке cityCenters_nearest тип float64, хотя диапазон чисел в этой колонке от 181 до 65968 - можем изменить тип на float32, а также изменим название колонки на city_centers_nearest
15. в колонке parks_around3000 тип float64, хотя диапазон чисел в этой колонке от 0 до 3 и хранятся по факту целочисленные значения - можем изменить тип на int8, а также изменим название колонки на parks_around_3000
16. в колонке parks_nearest тип float64, хотя диапазон чисел в этой колонке от 1 до 31900 и хранятся по факту целочисленные значения - можем изменить тип на int8
17. в колонке ponds_around3000 тип float64, хотя диапазон чисел в этой колонке от 0 до 3 и хранятся по факту целочисленные значения - можем изменить тип на int8, а также изменим название колонки на ponds_around_3000
18. в колонке ponds_nearest тип float64, хотя диапазон чисел в этой колонке от 13 до 1344 - можем изменить тип на float16
19. в колонке days_exposition тип float64, хотя диапазон чисел в этой колонке от 1 до 15800 и хранятся по факту целочисленные значения - можем изменить тип на int16

In [32]:
print(f'int8: {2**8}; int16: {2**16}, int32: {2**32}, int64: {2**64}')

int8: 256; int16: 65536, int32: 4294967296, int64: 18446744073709551616
